# Day 7

### Part 1

You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to **issues in luggage processing**.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:
```
light red bags contain 1 bright white bag, 2 muted yellow bags.  
dark orange bags contain 3 bright white bags, 4 muted yellow bags.  
bright white bags contain 1 shiny gold bag.  
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.  
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.  
dark olive bags contain 3 faded blue bags, 4 dotted black bags.  
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.  
faded blue bags contain no other bags.  
dotted black bags contain no other bags.
```

These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.

You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

- A bright white bag, which can hold your shiny gold bag directly.
- A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
- A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
- A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.

So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)

In [ ]:
debug = False
FILENAME="data/Day_07_input.txt"

In [ ]:
import re
from pprint import pprint
bags = {}

with open(FILENAME) as reader:
    for line in reader.readlines():
        line = line.strip()
        #if debug: print(line)
        m = re.match(r'(.*) bags contain (.*)', line)
        container = m.group(1)
        if container not in bags.keys():
            bags[container] = []
        #if debug: print("container: ", container)
        contains = []
        for content in m.group(2).split(', '):
            if content == "no other bags.":   
                break
            m = re.match(r'\S*(\d+) (.*) bags?\.?', content)
            color =  m.group(2)
            if color not in bags.keys():
                bags[color] = []
            #if debug: print("++", m.group(1),"-", m.group(2))
            bags[color].append( {container: int(m.group(1))}  )     

if debug: pprint(bags)
    

In [ ]:
to_search = ['shiny gold']
found = set()

while len(to_search)>0:
    color = to_search.pop()
    for item in bags[color]:
        for key in item.keys():
            to_search.append(key)
            found.add(key)

if debug: print(found)
print(len(found))

## Part 2

It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

- faded blue bags contain 0 other bags.
- dotted black bags contain 0 other bags.
- vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
- dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.

So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within **each** of those): 1 + 1*7 + 2 + 2*11 = **32** bags!

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

Here's another example:

- shiny gold bags contain 2 dark red bags.
- dark red bags contain 2 dark orange bags.
- dark orange bags contain 2 dark yellow bags.
- dark yellow bags contain 2 dark green bags.
- dark green bags contain 2 dark blue bags.
- dark blue bags contain 2 dark violet bags.
- dark violet bags contain no other bags.

In this example, a single shiny gold bag must contain 126 other bags.

**How many individual bags are required inside your single shiny gold bag?**


In [ ]:
bags = {}

# Fill up a lovely data structure with 
with open(FILENAME) as reader:
    for line in reader.readlines():
        line = line.strip()
        m = re.match(r'(.*) bags contain (.*)', line)
        container = m.group(1)
        contains = []
        # print("Container: {}".format(container))
        for content in m.group(2).split(', '):
            content = content.strip()
            if content != "no other bags.":
                m = re.match(r'\S*(\d+) (.*) bags?\.?', content)
                color = m.group(2)
                qty = int(m.group(1))
                # if debug: print("\tqty: {:-2}  color: {}".format(qty, color))
                contains.append({'qty':qty, 'color':color})
            else:
                pass
        bags[container] = contains

if debug: pprint(bags)

In [ ]:
def get_bags(color,depth):
    if debug: print("\t"*depth + "Bag: {} => {}".format(color, bags[color]))
    depth += 1
    count = 0
    for bag in bags[color]:
        # if debug: print("\t"*depth + "Qty: {:-2} Color: {}".format(bag['qty'], bag['color']))
        count += bag['qty'] + (get_bags(bag['color'], depth)*bag['qty'])
        if debug: print("\t"*depth + "Count:", count)
    # print("Current Count: ", count)
    return(count) # include a count of this color of bag

start = "shiny gold"
print(get_bags(start,0))

## Done!